In [1]:
%matplotlib inline
import matplotlib as mpl;
import matplotlib.pyplot as plt;

In [2]:
import numpy as np;
import gzip;
import StringIO;

def parse_header_of_csv(csv_str):
    # Isolate the headline columns:
    headline = csv_str[:csv_str.index('\n')];
    columns = headline.split(',');

    # The first column should be timestamp:
    assert columns[0] == 'timestamp';
    # The last column should be label_source:
    assert columns[-1] == 'label_source';
    
    # Search for the column of the first label:
    for (ci,col) in enumerate(columns):
        if col.startswith('label:'):
            first_label_ind = ci;
            break;
        pass;

    # Feature columns come after timestamp and before the labels:
    feature_names = columns[1:first_label_ind];
    # Then come the labels, till the one-before-last column:
    label_names = columns[first_label_ind:-1];
    for (li,label) in enumerate(label_names):
        # In the CSV the label names appear with prefix 'label:', but we don't need it after reading the data:
        assert label.startswith('label:');
        label_names[li] = label.replace('label:','');
        pass;
    
    return (feature_names,label_names);

def parse_body_of_csv(csv_str,n_features):
    # Read the entire CSV body into a single numeric matrix:
    full_table = np.loadtxt(StringIO.StringIO(csv_str),delimiter=',',skiprows=1);
    
    # Timestamp is the primary key for the records (examples):
    timestamps = full_table[:,0].astype(int);
    
    # Read the sensor features:
    X = full_table[:,1:(n_features+1)];
    
    # Read the binary label values, and the 'missing label' indicators:
    trinary_labels_mat = full_table[:,(n_features+1):-1]; # This should have values of either 0., 1. or NaN
    M = np.isnan(trinary_labels_mat); # M is the missing label matrix
    Y = np.where(M,0,trinary_labels_mat) > 0.; # Y is the label matrix
    
    return (X,Y,M,timestamps);

'''
Read the data (precomputed sensor-features and labels) for a user.
This function assumes the user's data file is present.
'''
def read_user_data(uuid):
    user_data_file = 'ExtraSensory.per_uuid_features_labels/%s.features_labels.csv.gz' % uuid;

    # Read the entire csv file of the user:
    with gzip.open(user_data_file,'rb') as fid:
        csv_str = fid.read();
        pass;

    (feature_names,label_names) = parse_header_of_csv(csv_str);
    n_features = len(feature_names);
    (X,Y,M,timestamps) = parse_body_of_csv(csv_str,n_features);

    return (X,Y,M,timestamps,feature_names,label_names);

In [3]:
def get_label_pretty_name(label):
    if label == 'FIX_walking':
        return 'Walking';
    if label == 'FIX_running':
        return 'Running';
    if label == 'LOC_main_workplace':
        return 'At main workplace';
    if label == 'OR_indoors':
        return 'Indoors';
    if label == 'OR_outside':
        return 'Outside';
    if label == 'LOC_home':
        return 'At home';
    if label == 'FIX_restaurant':
        return 'At a restaurant';
    if label == 'OR_exercise':
        return 'Exercise';
    if label == 'LOC_beach':
        return 'At the beach';
    if label == 'OR_standing':
        return 'Standing';
    if label == 'WATCHING_TV':
        return 'Watching TV'
    
    if label.endswith('_'):
        label = label[:-1] + ')';
        pass;
    
    label = label.replace('__',' (').replace('_',' ');
    label = label[0] + label[1:].lower();
    label = label.replace('i m','I\'m');
    return label;

In [4]:
import glob
uuid_list = glob.glob("ExtraSensory.per_uuid_features_labels/*.gz")
for (uuid, label) in enumerate(uuid_list):
    uuid_list[uuid] = label.replace('ExtraSensory.per_uuid_features_labels/','');
    pass;

for (uuid, label) in enumerate(uuid_list):
    uuid_list[uuid] = label.replace('.features_labels.csv.gz','');
    pass;
print uuid_list


['A7599A50-24AE-46A6-8EA6-2576F1011D81', '78A91A4E-4A51-4065-BDA7-94755F0BB3BB', '11B5EC4D-4133-4289-B475-4E737182A406', 'CDA3BBF7-6631-45E8-85BA-EEB416B32A3C', 'CF722AA9-2533-4E51-9FEB-9EAC84EE9AAC', '5152A2DF-FAF3-4BA8-9CA9-E66B32671A53', 'A76A5AF5-5A93-4CF2-A16E-62353BB70E8A', '665514DE-49DC-421F-8DCB-145D0B2609AD', '83CF687B-7CEC-434B-9FE8-00C3D5799BE6', '27E04243-B138-4F40-A164-F40B60165CF3', '1538C99F-BA1E-4EFB-A949-6C7C47701B20', '5EF64122-B513-46AE-BCF1-E62AAC285D2C', '7D9BB102-A612-4E2A-8E22-3159752F55D8', '33A85C34-CFE4-4732-9E73-0A7AC861B27A', '74B86067-5D4B-43CF-82CF-341B76BEA0F4', '3600D531-0C55-44A7-AE95-A7A38519464E', '9DC38D04-E82E-4F29-AB52-B476535226F2', '5119D0F8-FCA8-4184-A4EB-19421A40DE0D', '7CE37510-56D0-4120-A1CF-0E23351428D2', '1DBB0F6F-1F81-4A50-9DF4-CD62ACFA4842', '1155FF54-63D3-4AB2-9863-8385D0BD0A13', '61359772-D8D8-480D-B623-7C636EAD0C81', '24E40C4C-A349-4F9F-93AB-01D00FB994AF', 'B9724848-C7E2-45F4-9B3F-A1F38D864495', '0A986513-7828-4D53-AA1F-E02D6DF9561B',

In [5]:
import json
import os
import errno

count = 0

'''for uuid in uuid_list:
    (X,Y,M,timestamps,feature_names,label_names) = read_user_data(uuid);

    modified_labels = []
    for i in range(len(label_names)):
        modified_labels.append(get_label_pretty_name(label_names[i]))

    for i in range(len(timestamps)):
        dirname  = "extrasensory.labels." + str(uuid[0:8])
        filename = str(timestamps[i]) + ".server_predictions.json"
        fullname = "60TESTERS/" + dirname + "/" + filename

        if not os.path.exists(os.path.dirname(fullname)):
            try:
                os.makedirs(os.path.dirname(fullname))
            except OSError as exc: # Guard against race condition
                if exc.errno != errno.EEXIST:
                    raise

        data = {}
        data['label_names'] = modified_labels

        probabilities = []
        for ii in range(len(label_names)):
            bools = Y[0, ii]
            if bools == True:
                probabilities.append(1.0)
            else:
                probabilities.append(0.0)
        data['label_probs'] = probabilities

        with open(fullname, "w+") as f:
            json.dump(data, f)
    count = count + 1
    print count
    print "finished!" + fullname '''

1
finished!60TESTERS/extrasensory.labels.A7599A50/1447307917.server_predictions.json
2
finished!60TESTERS/extrasensory.labels.78A91A4E/1449809384.server_predictions.json
3
finished!60TESTERS/extrasensory.labels.11B5EC4D/1440612571.server_predictions.json
4
finished!60TESTERS/extrasensory.labels.CDA3BBF7/1442078275.server_predictions.json
5
finished!60TESTERS/extrasensory.labels.CF722AA9/1448236775.server_predictions.json
6
finished!60TESTERS/extrasensory.labels.5152A2DF/1443647130.server_predictions.json
7
finished!60TESTERS/extrasensory.labels.A76A5AF5/1443557065.server_predictions.json
8
finished!60TESTERS/extrasensory.labels.665514DE/1449782489.server_predictions.json
9
finished!60TESTERS/extrasensory.labels.83CF687B/1450117058.server_predictions.json
10
finished!60TESTERS/extrasensory.labels.27E04243/1448996921.server_predictions.json
11
finished!60TESTERS/extrasensory.labels.1538C99F/1445982789.server_predictions.json
12
finished!60TESTERS/extrasensory.labels.5EF64122/1442671439.s

In [74]:
import datetime;
import pytz;

# weekday_names = ['Mon','Tue','Wed','Thu','Fri','Sat','Sun'];
time_zone = pytz.timezone('US/Pacific');
one_week  = datetime.timedelta(days=7)
one_day   = datetime.timedelta(days=1)
one_hour  = datetime.timedelta(hours=1)

# you can change the timeframe
timeframe = one_week

for uuid in uuid_list:
    (X,Y,M,timestamps,feature_names,label_names) = read_user_data(uuid);
    print uuid

    start_time = datetime.datetime.fromtimestamp(timestamps[0],tz=time_zone)
    end_time   = start_time + timeframe

    weekday_str = weekday_names[start_time.weekday()];
    time_of_day = start_time.strftime('%I:%M%p');

    print "Begin at " + weekday_str + " " + time_of_day + " in one week" 
    
    
    modified_labels = []
    for i in range(len(label_names)):
        modified_labels.append(get_label_pretty_name(label_names[i]))

    counts_per_label  = np.zeros(len(modified_labels))
    actual_begin_time = start_time
    actual_end_time   = start_time
    if_not_begin = True
    
    for i in range(len(timestamps)):
        current_time = datetime.datetime.fromtimestamp(timestamps[i],tz=time_zone)
        if current_time >= start_time and current_time < end_time:
            counts_per_label = np.add(counts_per_label, Y[i, :])
            actual_end_time = current_time
            if if_not_begin:
                actual_begin_time = current_time
                if_not_begin = False

    labels_and_counts = zip(modified_labels, counts_per_label);
    sorted_labels_and_counts = sorted(labels_and_counts,reverse=True,key=lambda pair:pair[1]);
    print "start from ";
    print actual_begin_time;
    print "to"
    print actual_end_time;
    print "Actual time spent";
    print actual_end_time - actual_begin_time
    print "-"*20;
    for (label,count) in sorted_labels_and_counts:
        print "%s - %d minutes" % (label, count);
        pass;
    print "-"*20;

A7599A50-24AE-46A6-8EA6-2576F1011D81
Begin at Thu 02:24PM in one week
start from 
2015-11-05 14:24:57-08:00
to
2015-11-11 21:58:37-08:00
Actual time spent
6 days, 7:33:40
--------------------
Indoors - 2451 minutes
Sitting - 2420 minutes
Phone on table - 2049 minutes
At home - 1914 minutes
Surfing the internet - 1434 minutes
Phone in bag - 1387 minutes
At school - 770 minutes
Talking - 666 minutes
Walking - 507 minutes
Standing - 501 minutes
Drive - I'm the driver - 424 minutes
In a car - 388 minutes
Lying down - 352 minutes
Sleeping - 352 minutes
Outside - 299 minutes
Shopping - 263 minutes
Exercise - 242 minutes
Eating - 156 minutes
Strolling - 148 minutes
On a bus - 86 minutes
Phone in hand - 72 minutes
In a meeting - 44 minutes
Grooming - 30 minutes
Dressing - 30 minutes
Bathing - shower - 18 minutes
At a restaurant - 15 minutes
With friends - 15 minutes
At main workplace - 1 minutes
With co-workers - 1 minutes
Running - 0 minutes
Bicycling - 0 minutes
Lab work - 0 minutes
In class

A76A5AF5-5A93-4CF2-A16E-62353BB70E8A
Begin at Tue 01:06PM in one week
start from 
2015-09-22 13:06:46-07:00
to
2015-09-29 13:04:25-07:00
Actual time spent
6 days, 23:57:39
--------------------
Indoors - 4009 minutes
Phone on table - 3987 minutes
At home - 3645 minutes
Lying down - 2849 minutes
Sleeping - 2690 minutes
Sitting - 2472 minutes
At school - 1984 minutes
Phone in pocket - 1958 minutes
At main workplace - 1698 minutes
Lab work - 1351 minutes
Standing - 1207 minutes
Walking - 800 minutes
Eating - 459 minutes
Watching TV - 448 minutes
With co-workers - 437 minutes
In a car - 250 minutes
Drive - I'm the driver - 202 minutes
Phone in hand - 164 minutes
Exercise - 154 minutes
Surfing the internet - 153 minutes
At a restaurant - 145 minutes
At the gym - 98 minutes
Bathing - shower - 97 minutes
Toilet - 93 minutes
Running - 88 minutes
In a meeting - 81 minutes
Grooming - 76 minutes
Outside - 73 minutes
Drinking (alcohol) - 58 minutes
Bicycling - 56 minutes
Doing laundry - 56 minutes


7D9BB102-A612-4E2A-8E22-3159752F55D8
Begin at Mon 05:45PM in one week
start from 
2015-11-16 17:45:13-08:00
to
2015-11-23 12:55:50-08:00
Actual time spent
6 days, 19:10:37
--------------------
At school - 637 minutes
Sitting - 621 minutes
Phone on table - 540 minutes
Indoors - 477 minutes
Computer work - 342 minutes
With friends - 127 minutes
At home - 112 minutes
Surfing the internet - 83 minutes
Walking - 61 minutes
Watching TV - 52 minutes
Phone in pocket - 42 minutes
Phone in hand - 37 minutes
In a meeting - 24 minutes
Outside - 20 minutes
At a restaurant - 14 minutes
Strolling - 12 minutes
Toilet - 2 minutes
Standing - 2 minutes
Singing - 1 minutes
Grooming - 1 minutes
Lying down - 0 minutes
Running - 0 minutes
Bicycling - 0 minutes
Sleeping - 0 minutes
Lab work - 0 minutes
In class - 0 minutes
At main workplace - 0 minutes
In a car - 0 minutes
On a bus - 0 minutes
Drive - I'm the driver - 0 minutes
Drive - I'm a passenger - 0 minutes
Exercise - 0 minutes
Cooking - 0 minutes
Shopp

7CE37510-56D0-4120-A1CF-0E23351428D2
Begin at Wed 02:01PM in one week
start from 
2015-10-07 14:01:22-07:00
to
2015-10-14 09:36:53-07:00
Actual time spent
6 days, 19:35:31
--------------------
Indoors - 7462 minutes
Phone on table - 5717 minutes
At home - 5151 minutes
Sitting - 4743 minutes
Lying down - 3490 minutes
Sleeping - 3420 minutes
At school - 3189 minutes
Talking - 2186 minutes
At main workplace - 2085 minutes
With friends - 1500 minutes
With co-workers - 1353 minutes
Phone in hand - 1081 minutes
Computer work - 1017 minutes
Phone in pocket - 931 minutes
Watching TV - 928 minutes
Phone in bag - 887 minutes
Walking - 657 minutes
Standing - 512 minutes
Eating - 446 minutes
In class - 418 minutes
In a car - 302 minutes
At a bar - 299 minutes
At a party - 297 minutes
In a meeting - 296 minutes
Grooming - 130 minutes
On a bus - 95 minutes
At a restaurant - 92 minutes
Drinking (alcohol) - 92 minutes
Bathing - shower - 32 minutes
At the beach - 28 minutes
Toilet - 23 minutes
Dressing

0A986513-7828-4D53-AA1F-E02D6DF9561B
Begin at Tue 11:06AM in one week
start from 
2015-12-08 11:06:37-08:00
to
2015-12-11 10:48:27-08:00
Actual time spent
2 days, 23:41:50
--------------------
Sitting - 2253 minutes
Indoors - 2177 minutes
At home - 1784 minutes
With friends - 1648 minutes
Lying down - 1145 minutes
Sleeping - 1145 minutes
At school - 753 minutes
Phone on table - 553 minutes
Talking - 537 minutes
Phone in pocket - 300 minutes
Standing - 284 minutes
Walking - 190 minutes
Eating - 136 minutes
Bathing - shower - 107 minutes
Outside - 77 minutes
Dressing - 47 minutes
Toilet - 41 minutes
Stairs - going up - 25 minutes
Stairs - going down - 8 minutes
Running - 7 minutes
Exercise - 7 minutes
Bicycling - 0 minutes
Lab work - 0 minutes
In class - 0 minutes
In a meeting - 0 minutes
At main workplace - 0 minutes
In a car - 0 minutes
On a bus - 0 minutes
Drive - I'm the driver - 0 minutes
Drive - I'm a passenger - 0 minutes
At a restaurant - 0 minutes
Cooking - 0 minutes
Shopping - 

8023FE1A-D3B0-4E2C-A57A-9321B7FC755F
Begin at Tue 04:51PM in one week
start from 
2015-10-27 16:51:42-07:00
to
2015-11-03 15:51:34-08:00
Actual time spent
6 days, 23:59:52
--------------------
Indoors - 3893 minutes
At home - 3374 minutes
Lying down - 2551 minutes
Sitting - 2395 minutes
Sleeping - 1832 minutes
Standing - 1119 minutes
At school - 1096 minutes
Computer work - 818 minutes
At main workplace - 529 minutes
With friends - 401 minutes
Bicycling - 338 minutes
Exercise - 338 minutes
Talking - 307 minutes
In a car - 304 minutes
Drive - I'm the driver - 245 minutes
Walking - 242 minutes
Grooming - 206 minutes
Phone on table - 200 minutes
Phone in pocket - 187 minutes
Cooking - 169 minutes
Eating - 153 minutes
With co-workers - 152 minutes
Surfing the internet - 140 minutes
Cleaning - 115 minutes
Shopping - 106 minutes
Toilet - 104 minutes
Bathing - shower - 78 minutes
Drive - I'm a passenger - 59 minutes
In a meeting - 57 minutes
Phone in hand - 50 minutes
Phone in bag - 50 minute

098A72A5-E3E5-4F54-A152-BBDA0DF7B694
Begin at Tue 10:14AM in one week
start from 
2015-08-04 10:14:18-07:00
to
2015-08-08 13:25:53-07:00
Actual time spent
4 days, 3:11:35
--------------------
At home - 2106 minutes
Indoors - 2093 minutes
Lying down - 1817 minutes
Sleeping - 1742 minutes
Sitting - 1380 minutes
At main workplace - 1269 minutes
Computer work - 915 minutes
At school - 378 minutes
Standing - 158 minutes
Walking - 140 minutes
Surfing the internet - 129 minutes
In a car - 96 minutes
Exercise - 95 minutes
Lab work - 89 minutes
Drive - I'm the driver - 87 minutes
In a meeting - 81 minutes
Bicycling - 62 minutes
Running - 33 minutes
Toilet - 26 minutes
Cooking - 25 minutes
Eating - 25 minutes
Grooming - 16 minutes
Outside - 14 minutes
Phone in hand - 14 minutes
Dressing - 10 minutes
Drive - I'm a passenger - 9 minutes
At the beach - 8 minutes
Stairs - going up - 1 minutes
Stairs - going down - 1 minutes
In class - 0 minutes
On a bus - 0 minutes
At a restaurant - 0 minutes
Phone 

B09E373F-8A54-44C8-895B-0039390B859F
Begin at Thu 03:43PM in one week
start from 
2015-11-05 15:43:13-08:00
to
2015-11-12 10:26:05-08:00
Actual time spent
6 days, 18:42:52
--------------------
At home - 4656 minutes
Indoors - 2929 minutes
Lying down - 2378 minutes
Sleeping - 1482 minutes
Sitting - 1206 minutes
Watching TV - 1108 minutes
At school - 1045 minutes
Phone in pocket - 709 minutes
Computer work - 470 minutes
Standing - 402 minutes
Exercise - 308 minutes
Phone on table - 307 minutes
Talking - 266 minutes
Walking - 259 minutes
Drive - I'm the driver - 253 minutes
Phone in hand - 217 minutes
Outside - 190 minutes
Bicycling - 122 minutes
In class - 121 minutes
Running - 95 minutes
Eating - 77 minutes
Cooking - 30 minutes
At a restaurant - 21 minutes
In a meeting - 9 minutes
Lab work - 0 minutes
At main workplace - 0 minutes
In a car - 0 minutes
On a bus - 0 minutes
Drive - I'm a passenger - 0 minutes
Shopping - 0 minutes
Strolling - 0 minutes
Drinking (alcohol) - 0 minutes
Bathin

CA820D43-E5E2-42EF-9798-BE56F776370B
Begin at Tue 10:10AM in one week
start from 
2015-08-25 10:10:18-07:00
to
2015-08-31 15:54:00-07:00
Actual time spent
6 days, 5:43:42
--------------------
Indoors - 3287 minutes
Phone on table - 2738 minutes
Sitting - 2648 minutes
Lying down - 2541 minutes
At home - 2438 minutes
Sleeping - 2305 minutes
At school - 1396 minutes
Computer work - 1278 minutes
Phone in pocket - 918 minutes
Standing - 820 minutes
Walking - 352 minutes
Drive - I'm the driver - 280 minutes
Eating - 141 minutes
Phone in hand - 123 minutes
Cleaning - 117 minutes
Drinking (alcohol) - 74 minutes
Outside - 73 minutes
Talking - 48 minutes
At a restaurant - 33 minutes
Bicycling - 27 minutes
Exercise - 27 minutes
Bathing - shower - 20 minutes
Dressing - 4 minutes
Cooking - 3 minutes
Grooming - 2 minutes
Running - 0 minutes
Lab work - 0 minutes
In class - 0 minutes
In a meeting - 0 minutes
At main workplace - 0 minutes
In a car - 0 minutes
On a bus - 0 minutes
Drive - I'm a passenge

0E6184E1-90C0-48EE-B25A-F1ECB7B9714E
Begin at Mon 10:10AM in one week
start from 
2015-11-30 10:10:05-08:00
to
2015-12-07 09:24:48-08:00
Actual time spent
6 days, 23:14:43
--------------------
Indoors - 4549 minutes
Sitting - 3089 minutes
At home - 2494 minutes
Talking - 2438 minutes
With friends - 2418 minutes
Lying down - 1989 minutes
Walking - 1532 minutes
Sleeping - 1312 minutes
At school - 1069 minutes
Standing - 866 minutes
With co-workers - 840 minutes
Cleaning - 828 minutes
In class - 647 minutes
Outside - 546 minutes
At main workplace - 517 minutes
Eating - 376 minutes
Cooking - 331 minutes
Surfing the internet - 231 minutes
At a restaurant - 137 minutes
Dressing - 118 minutes
Grooming - 116 minutes
Phone on table - 107 minutes
Toilet - 96 minutes
Bathing - shower - 88 minutes
Stairs - going down - 87 minutes
Stairs - going up - 78 minutes
In a meeting - 63 minutes
Watching TV - 62 minutes
On a bus - 54 minutes
Shopping - 48 minutes
In a car - 42 minutes
Running - 0 minutes
Bi

In [75]:
print actual_end_time - actual_begin_time

5 days, 18:54:24


In [121]:
names = ['James', 'John', 'Robert', 'Michael', 'William', 'David', 'Richard', 'Joseph', 'Thomas', 'Charles', 'Christopher', 'Daniel', 'Matthew', 'Anthony', 'Donald', 'Mark', 'Paul', 'Steven', 'Andrew', 'Kenneth', 'George', 'Joshua', 'Kevin', 'Brian', 'Edward', 'Ronald', 'Timothy', 'Jason', 'Jeffrey', 'Ryan', 'Mary', 'Patricia', 'Jennifer', 'Elizabeth', 'Linda', 'Barbara', 'Susan', 'Jessica', 'Margaret', 'Sarah', 'Karen', 'Nancy', 'Betty', 'Lisa', 'Dorothy', 'Sandra', 'Ashley', 'Kimberly', 'Donna', 'Carol', 'Michelle', 'Emily', 'Amanda', 'Helen', 'Melissa', 'Deborah', 'Stephanie', 'Laura', 'Rebecca', 'Sharon']
print names

['James', 'John', 'Robert', 'Michael', 'William', 'David', 'Richard', 'Joseph', 'Thomas', 'Charles', 'Christopher', 'Daniel', 'Matthew', 'Anthony', 'Donald', 'Mark', 'Paul', 'Steven', 'Andrew', 'Kenneth', 'George', 'Joshua', 'Kevin', 'Brian', 'Edward', 'Ronald', 'Timothy', 'Jason', 'Jeffrey', 'Ryan', 'Mary', 'Patricia', 'Jennifer', 'Elizabeth', 'Linda', 'Barbara', 'Susan', 'Jessica', 'Margaret', 'Sarah', 'Karen', 'Nancy', 'Betty', 'Lisa', 'Dorothy', 'Sandra', 'Ashley', 'Kimberly', 'Donna', 'Carol', 'Michelle', 'Emily', 'Amanda', 'Helen', 'Melissa', 'Deborah', 'Stephanie', 'Laura', 'Rebecca', 'Sharon']


In [122]:
import datetime;
import pytz;
import json

# weekday_names = ['Mon','Tue','Wed','Thu','Fri','Sat','Sun'];
time_zone = pytz.timezone('US/Pacific');
one_week  = datetime.timedelta(days=7)
one_day   = datetime.timedelta(days=1)
one_hour  = datetime.timedelta(hours=1)

# you can change the timeframe
timeframe = one_week

index = 0

for uuid in uuid_list:
    (X,Y,M,timestamps,feature_names,label_names) = read_user_data(uuid);
    print uuid
    
    userName  = names[index]
    
    index = index + 1
    
    filename  = "Named_Analysis.json"

    userEmail = userName + "@ucsd,edu"
    avatar    = "https://api.adorable.io/avatars/250/" + userEmail + ".png"
    
    timeRecorded_days = 0.0
    
    start_time = datetime.datetime.fromtimestamp(timestamps[0],tz=time_zone)
    end_time   = start_time + timeframe

    weekday_str = weekday_names[start_time.weekday()];
    time_of_day = start_time.strftime('%I:%M%p');

    print "Begin at " + weekday_str + " " + time_of_day + " in one week" 
    
    
    modified_labels = []
    for i in range(len(label_names)):
        modified_labels.append(get_label_pretty_name(label_names[i]))

    counts_per_label  = np.zeros(len(modified_labels))
    actual_begin_time = start_time
    actual_end_time   = start_time
    if_not_begin = True
    
    for i in range(len(timestamps)):
        current_time = datetime.datetime.fromtimestamp(timestamps[i],tz=time_zone)
        if current_time >= start_time and current_time < end_time:
            counts_per_label = np.add(counts_per_label, Y[i, :])
            actual_end_time = current_time
            if if_not_begin:
                actual_begin_time = current_time
                if_not_begin = False

    labels_and_counts = zip(modified_labels, counts_per_label);
    sorted_labels_and_counts = sorted(labels_and_counts,reverse=True,key=lambda pair:pair[1]);
    timeRecorded_days = (actual_end_time - actual_begin_time).total_seconds()/60/60/24
    labels = {}
    for i in range(len(modified_labels)):
        labels[labels_and_counts[i][0]] = labels_and_counts[i][1]

    data = json.dumps(
        {
            userEmail:{
                "avatar": avatar,
                "userName": userName,
                "labels": labels,
                "timeRecorded(days)": timeRecorded_days
            }
        }
    )

    with open(filename, "a") as f:
        f.write(data)


A7599A50-24AE-46A6-8EA6-2576F1011D81
Begin at Thu 02:24PM in one week
78A91A4E-4A51-4065-BDA7-94755F0BB3BB
Begin at Tue 02:30PM in one week
11B5EC4D-4133-4289-B475-4E737182A406
Begin at Wed 09:07AM in one week
CDA3BBF7-6631-45E8-85BA-EEB416B32A3C
Begin at Wed 10:06AM in one week
CF722AA9-2533-4E51-9FEB-9EAC84EE9AAC
Begin at Mon 02:09PM in one week
5152A2DF-FAF3-4BA8-9CA9-E66B32671A53
Begin at Wed 02:17PM in one week
A76A5AF5-5A93-4CF2-A16E-62353BB70E8A
Begin at Tue 01:06PM in one week
665514DE-49DC-421F-8DCB-145D0B2609AD
Begin at Thu 01:22PM in one week
83CF687B-7CEC-434B-9FE8-00C3D5799BE6
Begin at Mon 09:58AM in one week
27E04243-B138-4F40-A164-F40B60165CF3
Begin at Mon 02:11PM in one week
1538C99F-BA1E-4EFB-A949-6C7C47701B20
Begin at Tue 03:14PM in one week
5EF64122-B513-46AE-BCF1-E62AAC285D2C
Begin at Wed 10:06AM in one week
7D9BB102-A612-4E2A-8E22-3159752F55D8
Begin at Mon 05:45PM in one week
33A85C34-CFE4-4732-9E73-0A7AC861B27A
Begin at Thu 02:04PM in one week
74B86067-5D4B-43CF-8

In [79]:
userName = "Tester-" + str(uuid[0:8])

In [111]:
import json

filename  = "Static_test_data.json"
userName  = "Tester_" + str(uuid[0:8])
userEmail = userName + "@test,com"
avatar    = "https://api.adorable.io/avatars/250/" + userEmail + ".png"

timeRecorded_days = (actual_end_time - actual_begin_time).total_seconds()/60/60/24

labels = {}
for i in range(len(modified_labels)):
    labels[labels_and_counts[i][0]] = labels_and_counts[i][1]

data = json.dumps(
    {
        userEmail:{
            "avatar": avatar,
            "userName": userName,
            "labels": labels,
            "timeRecorded(days)": timeRecorded_days
        }
    }
)

with open(filename, "w+") as f:
    f.write(data)

{  
   "angie@angie,com":{  
      "avatar":"https://api.adorable.io/avatars/250/angie@angie,com.png",
      "userName":"Angie",
      "labels":{  
         "key":2
      },
      "timeRecorded(days)":3.5
   }
}

In [89]:
filename  = "Static_test_data"
userName  = "Tester_" + str(uuid[0:8])
userEmail = userName + "@test,com"
avatar    = "https://api.adorable.io/avatars/250/" + userEmail + ".png"

timeRecorded_days = (actual_end_time - actual_begin_time).total_seconds()/60/60/24
data = {}
data[userEmail] = {}
userEmail['avatar']   = avatar
userEmail['userName'] = userName
userEmail['labels']   = {}
for i in range(len(modified_labels)):
    labels[labels_and_counts[i][0]] = labels_and_counts[i][1]
userEmail['timeRecorded(days)'] = (actual_end_time - actual_begin_time).total_seconds()/60/60/24

with open(fullname, "w+") as f:
    json.dump(data, f)

TypeError: 'str' object does not support item assignment